In [1]:
import pandas as pd
import geopandas as gpd

Učitavanje geopodataka

In [2]:
gdf = gpd.read_file("data/administrative_units/gradovi_opcine_zupanije.geojson")
gdf['Županija'] = gdf['text_right'].str.strip()
gdf['Grad/općina'] = gdf['text_left'].str.strip()

Učitavanje odabranih podataka iz popisa stanovništva

In [ ]:
census_percentages = []

bračni_status = pd.read_csv("data/census/bračni_status.csv")
bračni_status = bračni_status[bračni_status['Starost'] == 'Ukupno']
bračni_status_percentages = bračni_status.columns[16:26]
bračni_status_percentages =list(bračni_status_percentages)
census_percentages += bračni_status_percentages

br_djece = pd.read_csv("data/census/broj_živorođene_djece.csv")
br_djece = br_djece[br_djece['Starost'] == 'Ukupno']
br_djece_percentages = br_djece.columns[[17] + list(range(19, 30))]
br_djece_percentages =list(br_djece_percentages)
census_percentages += br_djece_percentages

dob = pd.read_csv("data/census/dob.csv")
dob = dob[dob['Spol'] == 'sv.']
dob_percentages = dob.columns[[14] + [18] +[23] + [25]]
dob_percentages =list(dob_percentages)
census_percentages += dob_percentages

ekonomska_aktivnost = pd.read_csv("data/census/ekonomska_aktivnost.csv")
ekonomska_aktivnost = ekonomska_aktivnost[(ekonomska_aktivnost['Spol'] == 'sv.') & (ekonomska_aktivnost['Starost'] == 'Ukupno')]
ekonomska_aktivnost_percentages = ekonomska_aktivnost.columns[[16] + [17] + list(range(19,25))]
ekonomska_aktivnost_percentages =list(ekonomska_aktivnost_percentages)
census_percentages += ekonomska_aktivnost_percentages

mjesto_rođenja = pd.read_csv("data/census/mjesto_rođenja_stanovanja.csv")
mjesto_rođenja = mjesto_rođenja[mjesto_rođenja['Spol'] == 'sv.']
mjesto_rođenja['Rođeni u gradu/općini stanovanja %'] = (mjesto_rođenja['U mjestu stanovanja'] + mjesto_rođenja['U drugom naselju istoga grada/općine2)']) / mjesto_rođenja['Ukupan broj stanovnika'] * 100
mjesto_rođenja_percentages = [col for col in mjesto_rođenja.columns[list(range(22,25)) + list(range(26, 33))] if col.endswith('%')]
mjesto_rođenja_percentages =list(mjesto_rođenja_percentages)
mjesto_rođenja_percentages.append('Rođeni u gradu/općini stanovanja %')
census_percentages += mjesto_rođenja_percentages

narodnost = pd.read_csv("data/census/narodnost.csv")
narodnost_percentages = [col for col in narodnost.columns[5:62] if col.endswith('%')]
narodnost_percentages =list(narodnost_percentages)
census_percentages += narodnost_percentages

područje_djelatnosti = pd.read_csv("data/census/područje_djelatnosti.csv")
područje_djelatnosti = područje_djelatnosti[(područje_djelatnosti['Starost'] == 'Ukupno') & (područje_djelatnosti['Spol'] == 'sv.')]
područje_djelatnosti_percentages = [col for col in područje_djelatnosti.columns if col.endswith('%')]
područje_djelatnosti_percentages =list(područje_djelatnosti_percentages)
census_percentages += područje_djelatnosti_percentages

položaj_u_zaposlenju = pd.read_csv("data/census/položaj_u_zaposlenju.csv")
položaj_u_zaposlenju = položaj_u_zaposlenju[(položaj_u_zaposlenju['Spol'] == 'sv.') & (položaj_u_zaposlenju['Starost'] == 'Ukupno')]
položaj_u_zaposlenju_percentages = položaj_u_zaposlenju.columns[[12] + list(range(14,19))]
položaj_u_zaposlenju_percentages =list(položaj_u_zaposlenju_percentages)
census_percentages += položaj_u_zaposlenju_percentages

vjera = pd.read_csv("data/census/vjera.csv")
vjera_percentages = [col for col in vjera.columns[5:28] if col.endswith('%')]
vjera_percentages =list(vjera_percentages)
census_percentages += vjera_percentages

završena_škola = pd.read_csv("data/census/završena_škola.csv")
završena_škola = završena_škola[(završena_škola['Spol'] == 'sv.') & (završena_škola['Starost'] == 'Ukupno')]
završena_škola_percentages = završena_škola.columns[list(range(15,20)) + list(range(21,25))]
završena_škola_percentages =list(završena_škola_percentages)
census_percentages += završena_škola_percentages

census_df = (
    bračni_status
    .merge(br_djece, on=['Županija', 'Grad/općina'], how='inner', suffixes=('bračni_status', '_br_djece'))
    .merge(dob, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_dob'))
    .merge(ekonomska_aktivnost, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_ekonomska_aktivnost'))
    .merge(mjesto_rođenja, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_mjesto_rođenja'))
    .merge(narodnost, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_narodnost'))
    .merge(područje_djelatnosti, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_područje_djelatnosti'))
    .merge(položaj_u_zaposlenju, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_položaj_u_zaposlenju'))
    .merge(vjera, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_vjera'))
    .merge(završena_škola, on=['Županija', 'Grad/općina'], how='inner', suffixes=('', '_završena_škola'))
)
df = gdf.merge(census_df, on=['Županija', 'Grad/općina'], how='inner')

print(census_percentages)

Učitavanje izbornih rezultata

In [ ]:
election_results = pd.read_csv("data/election_results/kombinirani_rezultati.csv")

party_percentages = election_results.columns[34:61] 
party_percentages = list(party_percentages)
print(party_percentages)

election_results['Županija'] = election_results['Županija'].str.strip()  # Strip any extra spaces
election_results['Grad/općina'] = election_results['Grad/općina/država'].str.strip()

df = gdf.merge(election_results, on=['Županija', 'Grad/općina'], how='inner').merge(census_df, on=['Županija', 'Grad/općina'], how='inner')

Na temelju podataka iz popisa i na temelju kojoj županiji pripada grad/općina, stvara se model koji pokušava predviditi izborne rezultate. Podatci se dijele na podatke za treniranje i podatke za testiranje.

In [213]:
features = census_percentages + ['Županija']
X = df[features]
X = pd.get_dummies(X, columns=['Županija'], prefix='Županija')

targets = party_percentages
Y = df[targets]

In [214]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

base_model = RandomForestRegressor(random_state=42)

# Multi-output wrapper
model = MultiOutputRegressor(base_model)
model.fit(X_train, Y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(random_state=42))

Metrike modela

In [215]:
# Evaluate on the training set
Y_train_pred = model.predict(X_train)
train_mae = mean_absolute_error(Y_train, Y_train_pred)
train_r2 = r2_score(Y_train, Y_train_pred)

# Evaluate on the test set
Y_test_pred = model.predict(X_test)
test_mae = mean_absolute_error(Y_test, Y_test_pred)
test_r2 = r2_score(Y_test, Y_test_pred)

# Print the results
print(f"Training MAE: {train_mae:.3f}, Training R²: {train_r2:.3f}")
print(f"Test MAE: {test_mae:.3f}, Test R²: {test_r2:.3f}")


Training MAE: 0.334, Training R²: 0.909
Test MAE: 0.889, Test R²: 0.360


Na kartama možemo usporediti stvarne rezultate i rezultate koje je predvidio model za testni dataset gradova/općina

In [ ]:
Y_test_pred_normalized = (Y_test_pred / Y_test_pred.sum(axis=1, keepdims=True)) * 100
error = Y_test - Y_test_pred_normalized
error_df = pd.DataFrame(error, columns=Y.columns)
df_with_errors = pd.merge(df, error_df, left_index=True, right_index=True)
for party_percentage in party_percentages:
    plot = df_with_errors.plot(
        column=f"{party_percentage}_y", 
        legend=True, 
        figsize=(15, 15)
    )
    plot.set_title(f"Actual result minus predicted result for {party_percentage}", fontsize=14)